In [1]:
class BitWriter:
    def __init__(self):
        self.buffer = []
        self.current_byte = 0
        self.bit_count = 0

    def write_bit(self, bit):
        self.current_byte = (self.current_byte << 1) | bit
        self.bit_count += 1
        if self.bit_count == 8:
            self.buffer.append(self.current_byte)
            self.current_byte = 0
            self.bit_count = 0

    def write_bits(self, value, num_bits):
        for i in range(num_bits - 1, -1, -1):
            self.write_bit((value >> i) & 1)

    def get_bytes(self):
        if self.bit_count > 0:
            self.buffer.append(self.current_byte << (8 - self.bit_count))
        return bytes(self.buffer)

class BitReader:
    def __init__(self, data):
        self.data = data
        self.byte_index = 0
        self.bit_index = 0

    def read_bit(self):
        if self.byte_index >= len(self.data):
            return None
        bit = (self.data[self.byte_index] >> (7 - self.bit_index)) & 1
        self.bit_index += 1
        if self.bit_index == 8:
            self.byte_index += 1
            self.bit_index = 0
        return bit

    def read_bits(self, num_bits):
        value = 0
        for _ in range(num_bits):
            bit = self.read_bit()
            if bit is None:
                return None
            value = (value << 1) | bit
        return value

def compress(data):
    writer = BitWriter()
    window_size = 4096
    lookahead_size = 16

    i = 0
    while i < len(data):
        # Find the longest match in the window
        best_length = 0
        best_distance = 0
        for j in range(max(0, i - window_size), i):
            length = 0
            while (i + length < len(data) and 
                   length < lookahead_size and 
                   data[j + length] == data[i + length]):
                length += 1
            if length > best_length:
                best_length = length
                best_distance = i - j

        if best_length > 2:
            # Encode a match
            writer.write_bit(1)
            writer.write_bits(best_distance - 1, 12)  # 12 bits for distance (up to 4096)
            writer.write_bits(best_length - 1, 4)     # 4 bits for length (up to 16)
            i += best_length
        else:
            # Encode a literal
            writer.write_bit(0)
            writer.write_bits(data[i], 8)
            i += 1

    return writer.get_bytes()

def compress_with_padding(data, target_size=None):
    """Compress data and optionally pad to target size with zeros."""
    compressed = compress(data)
    if target_size and len(compressed) < target_size:
        # Pad with zeros to reach target size
        compressed += b'\x00' * (target_size - len(compressed))
    return compressed

def decompress(compressed_data):
    reader = BitReader(compressed_data)
    result = bytearray()
    window_size = 4096

    while True:
        bit = reader.read_bit()
        if bit is None:
            break
        
        if bit == 0:
            # Literal
            byte = reader.read_bits(8)
            if byte is None:
                break
            result.append(byte)
        else:
            # Match
            distance = reader.read_bits(12)
            length = reader.read_bits(4)
            if distance is None or length is None:
                break
            distance += 1
            length += 1
            start = len(result) - distance
            for i in range(length):
                result.append(result[start + i])

    return bytes(result)

def compress_hex(hex_string):
    data = bytes.fromhex(hex_string)
    compressed = compress(data)
    return compressed.hex()

def decompress_hex(compressed_hex):
    compressed = bytes.fromhex(compressed_hex)
    decompressed = decompress(compressed)
    return decompressed.hex()

def print_compressed(compressed, chunk_size=256, do_reverse=False, pad_chunks=True):
    """Print compressed data in chunks, optionally padding chunks with zeros."""
    hex_string = ''.join(f"{b:02x}" for b in compressed)
    if do_reverse:
        hex_string = reverse_pairs(hex_string)
    
    for i in range(0, len(hex_string), chunk_size):
        chunk = hex_string[i:i+chunk_size]
        if pad_chunks and len(chunk) < chunk_size:
            # Pad the chunk with zeros to reach full chunk size
            chunk += '0' * (chunk_size - len(chunk))
        print(chunk)

def print_compressed_with_byte_padding(compressed, chunk_size=256, do_reverse=False):
    """Print compressed data padded to align with chunk boundaries."""
    # Calculate how many bytes we need to align to chunk boundaries
    bytes_per_chunk = chunk_size // 2  # Each byte = 2 hex characters
    remainder = len(compressed) % bytes_per_chunk
    
    if remainder != 0:
        # Pad the compressed data with zero bytes
        padding_needed = bytes_per_chunk - remainder
        compressed_padded = compressed + b'\x00' * padding_needed
    else:
        compressed_padded = compressed
    
    print_compressed(compressed_padded, chunk_size, do_reverse, pad_chunks=False)

def reverse_pairs(s):
    return ''.join(s[i:i+2][::-1] for i in range(0, len(s), 2))

def process_map(map_list, chunk_size, do_reverse, pad_mode="chunks"):
    """
    Process map with different padding options:
    - "chunks": Pad output chunks with zeros when printing
    - "bytes": Pad compressed data with zero bytes to align with chunk boundaries
    - "none": No padding
    """
    combined_compressed = b''
    piece_lengths = []

    for i, original_hex in enumerate(map_list):
        if i >= 1:  # Bottom part of the map
            original_hex = reverse_pairs(original_hex)
        
        original = bytes.fromhex(original_hex)
        compressed = compress(original)
        combined_compressed += compressed
        piece_lengths.append(len(compressed))
        
        print(f"Map piece {i + 1}:")
        print(f"Original length: {len(original)}")
        print(f"Compressed length: {len(compressed)}")
        print(f"Compression ratio: {len(compressed) / len(original):.2f}")
        print(f"Matches original: {original == decompress(compressed)}")
        print()

    print("Lengths of compressed pieces:")
    print(piece_lengths)
    print(f"Total compressed length: {len(combined_compressed)}")
    print("\nCompressed data:")
    
    if pad_mode == "chunks":
        print_compressed(combined_compressed, chunk_size, do_reverse, pad_chunks=True)
    elif pad_mode == "bytes":
        print_compressed_with_byte_padding(combined_compressed, chunk_size, do_reverse)
    else:
        print_compressed(combined_compressed, chunk_size, do_reverse, pad_chunks=False)
    
    print()

    return combined_compressed

# Example usage:
if __name__ == "__main__":
    # Example with padding
    test_data = b"Hello, World! This is a test string for compression."
    compressed = compress(test_data)
    
    print("Original compressed output:")
    print_compressed(compressed, chunk_size=32, pad_chunks=False)
    
    print("\nWith chunk padding:")
    print_compressed(compressed, chunk_size=32, pad_chunks=True)
    
    print("\nWith byte alignment padding:")
    print_compressed_with_byte_padding(compressed, chunk_size=32)

Original compressed output:
24194d86c378b04057379c8d86410880
a868349cc41002230880e865399d0407
33a1c8d26e33880cc6f39080c66f369c
0e465399ccd26f370b80

With chunk padding:
24194d86c378b04057379c8d86410880
a868349cc41002230880e865399d0407
33a1c8d26e33880cc6f39080c66f369c
0e465399ccd26f370b80000000000000

With byte alignment padding:
24194d86c378b04057379c8d86410880
a868349cc41002230880e865399d0407
33a1c8d26e33880cc6f39080c66f369c
0e465399ccd26f370b80000000000000


In [ ]:
# Example usage
map_1_2_top = """
07072a2a2a072a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a07075376765107712a0007567656545176505107712a000756505551565676505055505050504c4b505051765107075451565676505050504c4b505051565676517651070754504c072a2a2a2a2a2a2a2a2a2a2a2a004b5156567655505050555050555050504c
072a0707072a0707070700001d07070707071d070707070707646060706060522a002a536070606070606060522a002a53606060606060606060606060606b49476c60706060527660606060606060606b49476c60606060607060605276606b492a07070707710000001d0000002a576c606060607070606060606070606b49
2a07070700000007070700003c0d0d2a2d072a0771000771075f046f6f7e6f6f6f7f6f1b2b0404046f6f6f7e6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f046f7b604947606e4e6f6f6d6f6f6f6d6f6f6f6d6f60494760046f046f6f6f6f6f04046f605907077107000000002c3d0000000007437e04787f6f6f7e6d7e6f7f7e6f604a
2a07077100000007070700000707070715163d070700070707077e787a7a797a78797a7a1b1c6f6f7f797a7a7a7a7a7a787a7a7a7a7a6d7a7a7a6f040404605947606f7b7a7a6d7a7a7a6d7a7a7a7b6f604947606f7b04047a7a7a7a04046f606907077707000000001d007100000007077f79797a7a784d5e7a7a7e7b6f6149
2a07770d0d2d07070707000007710707252607070700070707077f797a786f6f6f6f7a7a7a7a7a7a7a7a7a6f6f6f7f6f7a787a7a7a7a6d7a7a7a6f6f6d6f606957606f7a7a7a5d6f6f6f5e7a7a7a7a6f604947606f6d7a7a6f6f6f7a7a7a6f605a0d0d3d07000071072a2a0000000007536f787a7879786d787879797f7e6049
2a070707071d0707070700000707072a0d3d2a7107000707077d7e7f6f6f7e04046f6f6f6f7f6f6f7f6f6f7e0404046f0b0c6f7a7a7a5d4e7a7a7a7a6d6f605a68606f7a7a7a6f7a7a7a7a6f4d6e6e6e604947606f6d7a6f047a7a6f7a046f604907710707000000072a2a0000000058617e7a7a7a7a6f6f6f7a0b797a7f604a
2a070707071d07070707000000002a3c2d070707070071072a7460606060607060604266464666437060606060606070606b6f047a7a6f6f6f6f7a0b5d6e604a68616f4d4e6f6d7a7a7a4d6e5e7a7a04604947606f5d6f7a7a307a046f4d6e60725050515654504c07071d0000710047607f797a7a6f7a6d7a6f1b7a786f6149
2a070771071d070707070000002a072a1d0707070700072a0707664646664440404107070771070744414666664445406361047a7a7a6f7a7a6f7a1b0c6f604947606e5e6d6f5d6e2e2f5e7a6f7a7a04604947606f7a6f04307a306e6f5e6f6b6060606060606b4907071d0000000047606f787a6f7a7a307a7a6f797a6f604a
4b505050517651505050505050505050505050505050505050505050505050505050505050505050505050507656545073606f7a7a7a6f7a7a6f6f6f6f6f605947606f7a5d6f7a7a3e3f7a7a6f7a7a6f604947606f7a6f6e6e3004046f7a0404046f6f6f6f6f6049072a2a0000070748617e797a6f7a3079304e7a79786f6049
476c60606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606c6f7a7a7a6f6f6f6f7a7a7a6f606a47606f7a7a6f7a7a6d7a7a7a6f4d6e6e604957606f7a7a6f046d7a6f047a6f6f6f046f6f7a6f60492c2a2a0000070747606f787a6f7a7a30795d6f6e6e6e614a
57606f6f6d6f6f7a0b2b046f6f6f6f6f6f6f6f0404046f6f6f6f6f6f6d6f6f1b1c6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f7a7a7a7a7a6f6d7a307a4d606a47606f7a7a7a6f6e5e7a7a6f7a6d7a6f604968606f7a7a046f6d6f7a7a7a6f6c60606b047a6f60493d07077100000047607f787a7a6f7a5d4e7e7a797a6f6059
68606f7b5e7a7a7a0b2b1c047a7a7a04047a7a0b0c7a7a7a7a7a7a4d5e7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a6f5d307a305e605a47606f7a7a4d5e6f6f6f6f7a4d5e7a6f604968616f7a7a7a7a6d7a7a7a7a6f60626360047a6f60725107070707544c47606f797a7a7a6f6f6f787a797a6f6069
68616f7a7a7a7a0b2b1c7a7a7a7a6f04046f0b2b0c6f6f7a7a6f6f6d6f6f6f6f6f6f6f0b0c6f6f7a7a6f6f6f6f6f6f6f6f6f6f7a7a7a7a7a6f7a7a307a7a604947606f6f7a6d7a7a7a7a7a7a6d7a7a6f604947606f6f7a4d6e5e7a7a046f6f604947606f7a6f6b6060520707536b49486079797a7a7a7a7a6d7a7a7a7978605a
68606f7a7a7a7a1b1c7a7a7a7a6f6c6060606060606b6f7a7a6f6c606060606060606060606b6f7a7a6f6c6060606060606b6f6f7a6f6f04046f6e5e7e7e604a47606f6f6f6d6f6f6f6f6f6f6d6f6f6f604947606f6f6f6d6f6f6f6f6f6f6f604947606f7a6f6f04047e6f7f6f605947607878797a78797a6d787a787979614a
58606f7b4e7a7a7a7a7a7a047a6f60624040404063606f7a7a6f6062404040404040404063606f7a7a6f606240404040636b6b6f7a6f6c606060606060606c49476b60606060606060606060606b6f6f6049476b60606060606060606060606c4947606f7a7a7a04047a787b6f606948606f78046f6f6f7e6d7f7e6f6f7f604a
48606f6f6d6f6f6f6f6f6f6f046f60494b50505073606f7a7a6f60725050504c4b50505073606f7a7a6f60725050504c5b63606f7a0b6162404146444045405c5b404040404646444542444063606f6f60495b404646444542444040404040405c47606f7a7a7a7a7a7a7a7a7f605a476b6070606060606b6f6f6c6060606c49
476b6060606060606060606060606c49476c6060606c6f7a7a6f6b6060606b49476c6060606c6f7a7a6f6b6060606b494b73616f7a1b6072505051565450504c00000007070707070707545073606f6f607250505050505050505050505050505073606e6e6e6e4e7a7a04046f60495b4040404040456360046f60624040405c
5b40404046664440404040404040405c47600404046f6f7a7a6f6d6f0404604947600404046f6f7a7a6f046f6f6f6049476c6c6f7a6f6b606060606060606b49077700076452565653537060606c6f6f6b60606060606060606060606060606060606c6f7a7a7a6d7a7a7a046f6072515656765050507360046f60725050504c
4b50505050505050505050517651505073600404046f7a7a7a7a5d6e0404604948606f7a7a7a7a7a7a7a7a047b6f604947606f6f7a6f6f6f6f042b2b04046049001d0007436f6f6f6f6f6f6f6f6f6f6f6f6f1b1c6f6f6f0404046f6f6f6f6f6f307a7a7a7a7a7a6d7a7a7a7a6f6b6060606060606060606c6f6f6b6060606b49
476c6060606060606060606060606060606c6f6f6f6f7a7a7a7a7a6f0b0b604948616f7a7a7a7a7a7a7a7a7a5d6e604957606f7a7a7a7a7a7a7a1b2b2b0c6059071d0007077a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a6f0404046f7a7a7a7a04307a7a7a7a7a7a6d7a7a7a7a6f6f6f6f6f0404046f6f6f6f6f6f6f6f6f6f6149
47606f6f6f6f6f04046f6d6f6f6f6f6f6f6f6f7a7a7a7a7a7a7a7a0b2b2b604947606f7b4e7a7a7a7a7a7a047a6f604968600c7a7a7a7a7a7a7a7a1b2b6f6049073c2d07077a7a7a7a7a7a047a047a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a30047a7a7a7a7a045d4e7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7b6f614a
47606f7b7a7a7a7a7a7a6d7a7a6f6f6f6f6f6f6f6f6f6f6f6f6f6f0b2b2b604957606f6f6d6f6f6f6f6f6f6f046f604968612b0c6f6f6f6f6f6f6f7a1b1c604907711d07536f6f6f6f6f0404046f6f6f6f6f6f6f6f0b0c6f6f6f6f6f6f6f6f306f7a7a6f6f6f04046d6f6f6f6f6f6f6f6f6f6f6f6f0404046f6f6f6f6f6f604a
47606e6e6e6e6f6f6f6f5e7a7a6f6c60606b6060606060606060606060606c49586b6060607060606060606060606c49586b606060606060606b6f7a7a6f604907001d077470606060706060606060606060606060606060606b6f04606263606f7a7a6f6c606060606060606060606060606060606060606042000043606c49
48606f7a7a307a7a7a7a307a7a6f60624040404046664440404040404040405c5b40406644404040404040404040405c5b4040404040404063600c7a7a6f605907001d0707664644404040404040404040404040404040406361046f605947606f7a7a6f6062404040464644454244404040404040404040410707710044405c
48606f7a6f6d7a7a7a7a7a6f7a6f60494b505050505176515050505050505050505050505050504c4b5050505051077107567656545050507360040c7a6f606a07071d07070707073c0d0d0d2d000000000000000000000073606f6f6a004760047a7a6f604900072a2a0000000000000000710000000000000000000000072a
48616f7a6f5d6e6f6f7a4d6e6e6e6049476c60606060606060606060606060606060606060606b49476c6060606052565360706060606060606c1c7a7a6f606907001d0707070707070707071d6452765352765656765360606c046f60655760047a7a6f604900072a2a0000000071000000000000000071000000000000072a
47606f7a6f7a7a6f6f6e5e6f7a6f604947600404046f6f6f6f6f6f6f6f0404046f6f6f6f6f6f604947606f6f04046f6f6d6f6f6f0404046f6f6f6f7a7a0b606907711d0707070707070707071d430404047e6f7f6f7e6f6d7f6f7a04044207436f7a7a6f60490007071d00004b5051565676505051765150505050505050504c
57606f7a6f7a7a6d7a7a7a6f7a6f6072736004047a7a7a7a7a306f6f7a7a7a7a7a7a7a7a6f6f604947606f7a0b0c7a7a6d7a7a7a7a7a7a7a7a7a7a7a7a1b604907001d0707070771070707071d686f7a79787a7a04787a6d797a7a7a04072c0d6f7a7a6f60490071071d0000476c607060606060606070606060606070606b49
68616f7a7a306e5e7a7a307a7a6f6b60606c6f7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a0b6f604947606f7a7a1b0c6f5e7a7a7a7a7a7a7a7a4d6e6e6e6e60490d0d3d0707070707070707071d437f7a7b7a7a0479787a5d6e6e7b7a6f0d3d076f7a046f60490007772a000047600404046f6d6f6f6f6f6f6f6d6f6f6f6f6059
68606f7a4d5e6f6f6f6f7a7a7a6f6f6f6f6f6f7a7a7a7a7a7a7a7a7a7a7a6f6f6f7a7a0b2b6f604947606f7a6f301b306f0c7a7a7a7a7a7a7a6d7a7a7a6f60490707070707070707070707071d677e786d7a0404787a7a797a7a78797e5207536f7a7a6f6049002c2a2a000047600404047a6d7a7a7a7a7a4d5e7a7a6f6f6049
58606f7b6d7a7a7a7a7a7a7a7a6f6f6f6f6f6f7a7a7a7a7a7a306f7a7a6f7a7a7a6f7a0b2b1c604947606e6e6e6e7b0b2b1c7a7a7a7a7a7a7a6d7a7a7a6f60490707070707070707077107073c537f6f6d6f6f6f7f7e6f6f7f7e6f7f7f6a58606f7a7a6f6049001d0707000047606f7a7a7a5d4e7a7a7a7a6d7a7a7a7a6f6049
48606f6f6d6f6f6f6f6f6f6f04046c60606b6f6f6f6f6f6f6f6f6f7a7a6f7a6f7a6f7a1b1c0460494760041b0c6f306f1c7a7a7a7a7a7a7a6f6d6f7a4d6e606907070707070707070707070707744246436646434246466643426646437547606f7a7a6f607250505050505073606e6e6e4e7a5d6f6f6f6f5e7a7a7a7a6f6049
""".replace('\n', '')

map_1_2_bottom = """
74b6060606060606060606060606c62636b606060606060606b6f6a7a7f6a7a7a7f6a7a740b70694740640c1c1a7f6a7a7a7a7a7a7a7a7f603e503e6e5f60696
70707070707070177070707070b4050515701770656765450505050505053706f6a7a7f6b60606060606060606c6f6a7a7d6a7f6a7a7a7a7f6a7a7a7a7f60695
b50404046466440404040404040404c5b5040404646644043606f6a7a7a7f6f6f6a7a7a740f606947406f6a7a7a7a7a7a7a7a7a7a7a7a7a7a7b7a7a7a7b006a5
7017707070707070707070707075c606062565350607060606060606060606c6f6a7a7f6f6f6f6f6f6f6f6f6f6f6f6a7a7d6f6d6a7a7a7d4e4f6a7a7a7f606a6
b405050505050515671505050505050505050505050505053706f6a7a7a7a7a7a7a7a7a7f64006947406f6a7a7a7a7a7a7a7a7a7a7a7d4e6e603f6a7b0b20694
7070707070a27070a2707070707034f6f6f6f6f6f6f6f640f6f640f6f64003c0c0b2c0a7a7a7a7a7a7a7a7a7a7a7a7a7a7d6f6d5e6e2f2e5d5f6e6e6e6e60696
74c60606060606060606060606060606060606060606060606c6f6a7a7a7a7a7a7a7a7a7a7f606947406f6a7a7a7a7a7a7a7a7a7a7a7d6a7a7f6a7b0b2b20694
7070707070705161707070c2d0d0d0f6a7a7a7a7a7a740a7a7a7a740a74003b1b2b1c1a7a7a7a7a7b0c0a7a7a7a7a7a7d4d5f6a7a7e3f3a7a7f6a7a7a7400696
7406f6b1b2b2c1f6d6f6f6f6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6a7a7a7a74040a7a7f6f606947406f6a7a7a7a7a7a7a7a7a7a7a7d6a7a7a7a7b140400695
7070707070705262707070d1701700f6a7a7a7a7a7a7a7a7a740a7a7a7a74003c1a7a7f6f6f6f6b0b2b2c0f6f6f6d4e6e5a7f6a7a7d6a7a7a7f6a7a7a7400694
7406f6b1b2b2c1a7d6a7a7a7a7a7a7d6a7a7a7a7f6f6f6f6f6f6f6f6f6f6f6f64040f6f6f6f606947406f6f6f6f6f6f6404040f6f6f6d6f6f6f6b0b2404006a5
7070707070a27070a2d0d0d3707035f6f6f6f6f6f6f6f6f640f6f64040f6f603f6a787f6c60606060606060606b6e5a7a7a7a7f6e6e5a7a7f6a7a7a7a7f60695
8406f6a7b1c1f6f6d6f6f6f6f6f6d4e5f6f6a7a7a7f6c6060606060706060606060606060606c69475b6b6f6f7c606060606060606060606060606060606c694
7070707070707070707070707085b606060606060606060606060606060606b6f6a797f606260404040404043606f6a7a7a7d4e5f6f6f6f6a7a7d4e6e6e60696
8416f6a7a7f6f6a7d5e4a7a7a7a7d6a7a7f6f6a7a7f60626046644040404040404040404040404c5176416f6f6062614646454040464664404045466044404c5
70700000000000000000000000b5040404646444542444040404040404043606e797a7f706957070d17070707406f6a7a7a7d5e4a7a7a7a7a7a7d6a7a7f606a5
7406f6a7f6f6a7a7a7d6a7a7a7a7d6a7a7a7f6f6a7f60694b4050555050505551570706567701770008606f6e7069400000000001700007017707070d11770a2
a2000000000000000000000000b405050505050505c47000d170707000007416e7a7a7f624707070d17070177506f6a7a7a7a7d6a7a7a7a7a7a7d6a7a7f60696
7506e6e6e6e6e4a7a7d5f6f6f6f6e5a7a7a7a7f6a7f6069474c6060606060606062535060735567077701640e7062715771770707070707070707070d1707070
b405051567156567654505050537c60606060606b694700077a2707070007516f7a7a7e725707070a2a270707606f6f6a7a7a7d6a7a7a7a7a7a7d6a7f6f60696
7406f6a7f6a7d6a7a7f6a7a7a7a7f6a7a7a7a7f6a7f606948406f6f6f6d6f6e7f7f6f640404006a57046c640f6b60606072567350607567070707000a2a27070
74c6060606060607060606060606c6f640f6b7f606940000a2a2707070001786e78797f7a6701770a2a270708506f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f606a5
7406f6a7f6a7d6a7f6d6a7a7a7a7a7f6a7a7a7f6a7f606947406e7b7a7d6a7a7a7d4e6e6e6e606a67034f6f6f6f6f6f7f6e7f6404040f47070707000a2a27070
7406f6f6f6f6f6f6f6d6f6f6f6f6f6f6a74040f60694000000d1001770707047240000345700707070d1707074b606070606060606060706060606060606c694
7406f6a7f6a7d6a7f6d5e6e2f2a7d4f6e6e6e6e6e6e606948406e6e5a7d6f6f6f6e5b0c0c0f616941785f687a7a7a7a7a7a7a7a797e770707017700070d17070
7506f6a7a7a7a7a7a7d6a7a7a7a7a7a7a7a740f606940000a2a2000000700000000000000000707070d17070b5040404040404040404040404040404040404c5
7406f6a7f6d4d5e6f6a7a7e3f3e6e5f6a7a7a7a740400694750640a7a7f6a7a7a7f6b1b2c1f706a47084408797a7a7a7a79797a7a7a7707070707000c2d37070
8606e6e6e6e4f6f6f6e5b0c0a7d4e6e6e6e6e5f606947000a2a20000007070007070707070177070a2a27070b4051565656715701770656765051567150505c4
7406e6e6e6e5a7a7f6a7a7d6a7a7a7f6a7a7a7f640400694860640a7f6d6a703a7a7f6b1c1f606a670864040f6f6f6f7f6f6e7f6e7f6c77070707000d1707070
8616f6a7a7f6a7a7b0f6b1c0a7d6a7a7b0c0a7f606967070c3d0d2007070707070177070707070c2a2a2707075c606060606062565350607060606060606b694
7506f6a7f6a7a7a7a7f6e6e5a7a7f6a7a7a7a7f6a7f606948616f7a7f6d5039703d4e6e6e6e616967047246466346664340706060606577070707000a2a27070
7606f6a7f6d6c003b1c0f6c1a7d6a7b0b2b2b2f6f470707070a2a2707017707070707070707070d1707070707034f6f7e7f6404040f6e74040f640f6e7f60694
76064040f6a7a7a7d4e5f6f6f6f6a7a7d4e6e6e6e6e606948606f6a7f6a7a70397e5f6a7a7f606967070177070707070706664666464001770707000a2a27070
7606f6a7f6d503a703e6e6e6e6e5b0b2b2b2c1a77070177070a2a2707070707070707070707070d1707070700000e7a7a7a7a7a787a7a7a7a7a7a740b7f60695
86064040f6f6a7a7d5e4a7a7a7a7a7a7d6a7f6f6b0c006948506f6a7a7f6e6e5a7e7a7a7a7f7f470707070707070707070707070700000707070707070d17070
8506f6a7f6a7a703a7a7f6a7a7a7b1c1b1c1a7a77070000000d100000046256535560000001700c3a2a200000000f687a7a7a7a7a7a7a7a7a7a7a7a7d5e60694
76064040a7f6f6a7a7d6a7a7a7a7a7a7d6f6f6b0b2c106948406f6a7d4e5f6f6f687a7a787977070177070701770707070707017700000707070170070d17070
7406f6c0a7f6e6e5a7f6a7a7a7a7a7a7a7a7a7a77070000000c3d2000086e7e7f7a6000000000000a2a200001700f6a7a7a7a7a7a7a7a7a7a7a7a7a7a7a70695
8506f6a7a7a7f6f6f6d6f6f6f6f6f6f6d6f6a7b0c1f606948416c0a7d6a7a7a7a7a7a7a7a7a770707070707070000000000000707070a2707070000070d17070
7506f6b2d4e5f6f6f6a7a7b0b0c0a7f6f6f6f6f6c77070707070d1707076f7b7f6a670007070707070d170700000f6c0a787a7a7a7a7a7a7a7a7a7a7a7a70696
7406f6b7a7a7d4e6e6e5a7b0b0c0a7a7d5e4b0c1b7f606947406b2c0d6a797a7a7a7a7a7a79770707070707070c2a2d0d0d0d2707070d17070a2a27070d17070
7606f6c1d6a7a7a7a7a7b0c1b1b1c0f6a740404006a57070a2a2d3177086f7e7e72570007070a2a270d170707000e7b1c0a7a7b7e4a7a7a7a7a7a740a7f606a5
7406f6f6f6f6d6f6f6f6b0b2b2b2c0f6f6d6b1f6f6f606947506b2c1d6f6e7f6f6f6f7e7f7e7c7707070a2d0d0d370707070c3a2d0d0d3d0d0a2a2d0d0777070
8506f6f6d6f6f6f6f6b0c1f6f6f6f6f64040404006947070a2a2707070472417345770007070a277d0d370707035f7f6e7f7f6f6d6f6e7f6f6f6e7f640f60694
74b60606060606066606060606060606060606060606c69470470606060706060606060706065770707070707070177070707070707017707070707070707070
74b6060606060606060606060606060606060606c69470707070707070707070707070707070d1707070707085b606060606060606060606060606060606c694
b504040404040414174414666664644454244404040404c5177034246464666466641464666470a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2
b50404040404040404646444542444040404040404c5a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2b5040404646444542444040404040404040404c5
""".replace('\n', '')

map_3_4_top = """
4b50515676545550515656765056515654555050505551565156545550504c2a2a2a2a2a2a2a2a2a4b505050515056505654555050505050565056545550504c072a2a2a2a2a2a2a4b5076555150515676545550505050765050555050507650505055555050515056505654555051567654555550765050505076507650504c
476c6060707060606060606060606060606060606060606060606060606b49070707071d07070700476c60606060606060606060606060606060606060606b492a07070707070707686c607060606060707060606060607060606060706060606060607060606060606060606060606070706060606060706060606060606b49
47606f6f6f6f6f6f6f6f6f6f6f6f7a7a7a6f6f6f6f6f6f6f6f0404046f60490d7707071d070777004760040404046d04046e6e6f6f6d6f6f6f6f04046f6f6049070707070707070748606f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6f6f6049
47606e7b4e7a7a7a7a7a7a7a7a6f7a7a7a7a7a7a7a7a7a7a6f0404046f6049070707071d71073c0d47600404047a6d7a7a7a7a7a4d5e7a7a7a7a04047a6f6049070707070707070747617e7a7a6d7a7a7a7a7a7a4d5e7a7a7a6d7a7a7a6f6f6f6f6f6f6f6f6f6f6f6f6f7a7a7a7a7a6d7a7a7a7a7a7a7a7a7a6d7a7a6f6f6169
47606f7a5d4e6f6f6f6f7a7a7a300b0b0c7a7a7a7a7a7a7a6f6f6f6f6f6069070707071d0707070048616f7a7a7a6d7a7a7a7a7a6d7a7a04047a7a7a7a6f6069070707710707070757607e7a795d4e6f6f6f6f6f6f6f6f6f6f6d7a7a7a6f6c6060706060606060606b6f7a7a7a7a7a6d7a7a7a7a7a7a7a7a7a6d7a7a7a6f6049
48616f7a7a6f7a7a7a7a6f7a7a6f1b2b04040c7a7a7a7a7a7a7a7a7a6f606a070707072a2a07070768606f7a7a7a6d7a6f6f6f6f5e7a7a04047a7a7b7a6f606a070707070707070707436f78797a6f7a7a7a7a7a6d7a7a7a7a6f4e7a7a6f60624040404040404063606f7a7a7a7a7a5d4e6f6f6f6f6f6f6f6f6d7a7a7a6f6049
68606f7a6f6d7a7a7a4d4e6f7a6f0b0404041c7a7a7a7a7a7a7a7a7a6f6169070707072a2a07070748606f7a7a7a5d6f7a7a7a6d6f7a7a7a7a7a7a7a7a6f61690707070771070707070779797a6f7a7a7a7a7a4d5e7a7a7a7a7a6f7a7a6f60494b50505050504c47606f6f6f6f7a7a7a6f7a7a7a7a7a7a7a7a6f4e7a7a6f6149
48606f7a6f6d7a2e2f5e5d6e4e6f1b2b041c7a7a7a7a7a7a7a7a7a797e606a07070771071d07070747616e6e4e7a6f7a7a7a4d5e4d6f7a7a4d6e6e6e6e6e606a0707070707710707070779797e6d7a7a7a7a7a6d7a7a7a7a4d4e7a6f7a6f6049476c6070606b49476b60606b6e6e4e6f6d7a7a4d4e7a7a7a4d4e6f7a7a6f6069
47616f7a6f5d6e3e3f7a7a6f6d6f7a1c7a7a7a7a7a7a7a7a7a797a797e606a07070707071d07070748606f7a6d7a6f6e6e2e2f5d5e6f6e6e5e6f6f6f6f6f606a0707077107070707070778786f5d6e2e2f6e6e2e2f7a7a2e2f5d6e6e6e6e604947606f6f6f60495b404063606f7a6d6f5d6e2e2f6d7a2e2f5e5d6e6e4e6f6049
48606f7a6f7a7a7a6d7a7a6f6d6f7a7a7a7a7a7a7a6f6f6f6f7e797a6f604907070707071d07070768606f7a5d6e6f7a4d3e3f7a7a6f7a7a7a6f6c6060606c490707070707070707075379787f7a7a3e3f7a7a3e3f7a7a3e3f7a7a6f7a6f604947606f7b6f607250504c47606f7a5d6f7a7a3e3f6d7a3e3f7a7a6f7a6d6f6049
68606f7a7a6f6e6e5e7a6f7a6d6f7a797a7a7a7a7a7e6c6060420707436c4907072a2a0d3d07070768606f7a7a7a6f7a6d7a6d7a7a6f7a7a7a6f60624040405c070771070707070758607e786f7a7a7a6d7a7a6d5d6e6e5e7a7a7a6f7a6f606947606f6d6f6b60706b4947606f7a7a6f7a7a6d7a5d6e5e7a7a7a6f7a6d6f6149
68606f7a4d5e6f6f6f6f7a4d5e307a7a797a797a0b0b60624107070707445c07072a2a070707070758606f7a7a7a4d6f5e7a5d4e6f7a7a7a4d6e60590707072a070707070707070768617e7a7a7f7a7a6d7a7a6d7a7a7a7a7a7a6f7a7a6f606a47606f5d4e6f6f6f604947606f7a7a7a6f6e5e7a7a7a7a7a7a6f7a7a6d6f6069
58606f7b5e7a7a7a7a7a7a6d7a6f797879797a0b2b2b60490707070707070707071d07070707074b73606f7a7a4d5e7a6f6f6f6f7a4d6e6e5e6f60690777712a070707070707070748606f7a7a7a6f6e5e7a7a6d7a7a7a7a7a6f4d6e6e6e616947607a0b6d7a7a6f606947606f7a7a4d5e6f6f6f6f6f6f6f6f7a4d6e5e6f6049
47606f6d6f6f6f6f6f6f6f6d6f6f7a787a797e1b2b2b60490707077107070707071d0707070707476c6c6f6f7a6d7a7a7a7a7a7a7a6d7a7a6f6f606a071d072a070707077107070748607e7a4d6e5e6f6f6f6f6f6f6f6f6f6f7a6d7a7a6f606a47600b2b5d4e7a6f606a47606f7a7a6d7a7a7a7a7a7a7a7a7a7a6d7a7a6f6069
476b60606060606060606060607042070707437060606c490707070707070707073c2d0771070747606f6f6f6f6d6f6f6f6f6f6f6f6d6f6f7b6e6069071d072a070707070707070748616f7a6d7a7a7a7a7a7a6d7a7a7a7a7a4d5e7a7a6f606a47601b1c7a6d7a6f616947606e6e6e5e7a7a7a7a7a7a7a7a7a7a6d7a6f6f6149
5b404040454166464646444045410707070707444066405c070707070707070707071d0707070768616f7a7a7a30303030303030303030303030605a073c0d2a070707070771070768606f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f604947606f7a04047a0b606a47606f6f6f6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6049
2a0707070707071d0707070707070707070707070707072a070707070707070707072a2a07070748606f7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a6f616a0707072a2a2a2a0707070707476b60607060606060606070606060706060606060606c4947606f7a04040b2b606a476b6060606070606060606060706060606060606c49
2a0707070707071d0707070707070707070707070707072a070707070707070707072a2a0d0d0d48606f7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a6f605a7107070707072a07070707075b404040404040404166406641664066444040464040405c47606f7a04041b2b60495b40404166406644404041664066444040464040405c
2a07072a2a0d0d3d0707070707070707077107070707072a07070707070707070707070707070747606f6f6f6f6f6f6f6f6f6f6f6f6f6f6f7a6f604a0707070707072a0707070707076476766507074b5050515056505654555050505050505073606f7a04047a6f60725050515676545550565056545550545550505050504c
2a07072a2a710707070707710707070707070707070707072a0707070707070707070707070707476b606060607060606060606060606b6f7a6f606a0707070707072a070707070707436f7b6a0d0d476c606060606060606060606070606060606c6f7a7a6d7a6f6b6060606060707060606060606060606060606060606b49
2a0707071d070707070707070707070771070707070707072a07070707710707070707070707075b404041664066444040464040406360047a6f60690707070707072a710707070707077e7e6a070757616f7e6f6f6f0b2b2b0c6f6f6f6d6f6f6f6f6f7a0b6d7a6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f6f6f6069
2a0707071d070707070707070707070707070707070707072a0707070707070707070771070707074b5055505050555656545050507360047a6f605a0707070707072a070707710707076f6f6a070707437f6f7a7a7a1b2b2b1c7a7a4d5e7a7a7a7a7a7a1b6d7a7a7a7a5d4e7a7a7a7a7a7a7a7a7a7a7a6d7a7a7a7a6f6f6049
2a0707711d070707070707070707070707070707070707072a070707070707070707070707070707576c6060606060606060606060606c6f7a6f614a0707070707072a070707070707536f6f6a0707070779787a040404041c7a7a04047a047a4d6e040404047a7a7a040404047a7a7a04047a047a7a7a047a7a7a7a7a6f6049
2a0707071d070707077107074b50555050505156765455504c0707072a070707070707070707070707437e6f7f7f6f7f6f6f6f6f6f04046f7a6f604a0707072a2a2a2a070707070758607f7f6a0707070779797a040404047a7a7a040404046e5e7a040404047a7a7a040404047a7a7a040404047a7a7a7a7a7a7a7a7a6f6169
2a0d0d0d3d07070707070707486c6060606060607070606b4907072a07070707070707070707070707077e7a7a7a7a7a7a7a7a7a7a7a047a7a6f60490771072a070707070707070748617e6f6a070707077978787a6d7a7a7a7a7a7a7a7a7a7a7a7a7a7a6d7a7a7a7a0b2b2b0c7a7a7a7a6d7a7a7a7a7a7a7a7a7a7a7a6f6049
2a070707070707070707070757606f7e7e6f7f6f040404604907070707070707070707070707070707077e7a7a7a7a7a0404047a7a7a7a7a7a6f604a0707072a070707070707075473607f6f6a070707537e79797a5d4e7a7a7a7a7a7a7a7a7a7a7a7a7a6d7a7a7a7a1b2b2b1c7a7a7a4d5e7a7a7a7a7a7a7a7a7a7a7a6f6049
2a070707070707070707710707437e7a7a7a7a6f040404604907070707070707070707710707070707536f7e6f7f6f6f040404046f6f6f6f6f6f61490707072a0771070707646070606c6f6f6a070758606f7978040404047a4d6e040404047a7a7a7a0404047a7a7a040404047a4d6e040404047a7a6f040404047a046f6049
2a0707070707070707070707000078787a7a7a6f6f6f6f604907070707070707072c0d0d0d2d0707586b606060606060606060606060706060606c490707072a0707070707686f6f7f7e6f7e6a2d0748617e7a790404046e6e5e7a04047b047a7a7a040404047a7a7a047a04046e5e7a040404047a0b0b047b04047a04046049
2a07070707070707070707070707787a7a7a307a7a7a6f60690d0d2d07070707071d0707073c0d0d5b4045406643454045464666444040404040405c0707072a0707070707677f7e7e7e7f6f6a3c0d48607e7f7a7a6d7a7a7a7a7a7a7a0b0c0c0c7a7a7a6d6f7a7a7a7a7a7a7a7a7a7a7a7a7a7a0b2b2b0c7a7a7a7a7a6d6069
2a07070707070771710707070707797a7a307a307a7a6f606a07071d07070707071d07070707070707070707070707070707070707070707071d07070707072a0707070707686f6f6241466675070068616f6f7e4d5e6f6f6f6f6f6f6f1b2b2b1c6f6f6f5d4e6f6f6f6f6f6f6f6f6f6f6f6f6f6f1b2b2b1c6f6f7a7a4d5e6049
2a077107070707070707070707536f797a7a307a7a7a6f616907073c0d0d2a07072a07070707070707070707070707070707070707070707071d07070707072a072c0d0d0d686f7f7251765365070748606060606060606060606060607060606060606060606060606060606060706060606060606060606b04046e5e6f6069
2a070707070707070707070758607e7a7a7a7a7a7a7a6f606a07070707070715160707070707070771070707070707070707070707710707071d07070707072a0d3d070707686f7e7f7f6f7e6a07075b404040404040404040404166406644404046404040404040404040416640664440404640404040636004047a7a6f6169


""".replace('\n', '')

map_3_4_bottom = """
a27070707070b40555550505370603030303a7a7a7a7f606a67070707070705262707070707070707070707070701770707070707070707070c3d0d0777070a2
707070177086f6e7e7e7e7f7a67070b40515655565450515056505654555050515656505156515656745550505156505654505556545c47406f640a7a7f60694
a2a2a270707074c60607070606c6f6a7404003a7a74040069470707070c2a27070a2d270707070707070707070707070707070707070707070177070707070a2
707070707047060706b6f6f69670c284c6060607060606060606060606060607060606060606060607070607060606060606060606b6957416f6404040f60694
a2a2a27070707406f6404040f6f6f6a7a7a74003030303069470707070d100707070d170707070707070707070707070707070707070707070707070707070a2
70707070700000443606f6e7a6d0d37416f6f6e7e7f6f6404003b2b2b2c1f6f6f6f6f6f6f640404003f6f6404040f6f6d6c1c1f6f606a6740640a703a7f60694
a270d17017707406f6a7a7a7a7a7a7a7a7a7a7a797a7f6069470707070d100707070d170b40505055515707070704555050505551565674555056545550505c4
70707070700000708416f7f79670707506e78797a7a7a7a7404003b1c1a7a740a7a7a7a7a7a740b10303a7a7a7a7b0a7d603a7a7f616967406f603a703e60696
a270d17070708416f6a740a7a7a7a7a7a7a7a797a797f7069470707070d100707070d17074c6060607062570703507060606070606060707060606060606b694
70701770707070707516f6f79670707034878787a7a7a7a7a7a7a703b0a7a7a7a7a703c0a7a7a7a7b103a7a7b040b2c003a703a7f606947406f6b703a7400694
a270d17070708416f6a7a7a7a7a7f6f6f6f7f6f7a797f7069470707070c3d2707070c3d07406f6f6f6f7f69787f7f6f6f6f6f6f6f6f6f6f6404040f6f6f60694
70707070707070000086f6e7a67070707087a78797a7a7a7a7a7b0c003a7a7a7a7a7a703c0a7a7a740c003a7b040b2c1c103a7a7f606a67406f6f64040400694
a2d0d37070708606c0a7a7a7a740f6c606060624707034c6947070707070d170707070708416f603a7a7979787a7a7a7b0a7a7a7a7a7a7a7404040a703f61694
7070707070a270000086e7e7a6707070708797a787a7a7a7a7a7b0b203a7a7a7a7a7a703b0a7a7a7a7b103a7b1b2c1b0b0c0a7a7f6069674b60606060606c694
a2707070a2708406b2c0a7a7a7a7f6062604147070707044c57070707070d170707070708606f6a7a7a7a797a7a7a7b0b2c0a7a7a7a7a7a7a7a7a7a7a7f60694
70707070a270a2707076e7e7a670707070878787a7a7a7a7a7a7b0c103a7a7a7a7a7a70340a7a7a7a7a70303a7a7a7b1404040c0f60694b504146444545404c5
a2707070a2a27416b2c1a7a7a7a7f6069470707070707070707070177070c3d0d0d0d0d08406f6a740a7a7a7a7a7a7b1b2b2c0a7a7a7a7a7a7a7a7a7a7f60627
0505c4c2d0a270707086f7f6a670707035f68797a7a74040a7a7b103c0a7a7a7a7a740034040c0a7a7a7b1c003a7a7b14040b240f606960070707070707070a2
a270177000a28406b2c0a7a7a7a7f60694701770707070707070707070707070177070707416f6a7a7a7a7a7a7a7a7a7b1b2b2c0a7a7a7a7a7a7a7a7a7b0b606
06b694d1707070707047e7f65770708506f7a7a7a7a74040a7a703b0b2c0a7a7a7a74003b0c1a74040a7a7b10303a7a7b1b1b140f606a60070177070707070a2
a270707000708606b2c1a7a7a7a7f606270515656745550505c4707070707070707070708406f6a7a7a7a7a7a7a7a7a7b1b2c1c1a7a7a7a7a7a7a7a7a7b1b240
b206a4a2a2a2a2a2a2000000007070841640a7a7a7a7a7a7a703b0c1b1c0a7a7a7a74003c1a7a7a740a7a7a7a7a7a7a7a7a7a7a7f616960070707077707070a2
a270707070708606c1a7a7a7a7a7f6b60606060607070606b694707070707070707070708606f6a7a7a7a7a7a7a7a7a7a7b1c1a7a7a7a7a7a7a7a7a7a7b0c0b2
b2069670707070000000000000707086064040a7a7a7a7a7a703b1b7b1b2a7a7a7a70303a7a7a7a7a7a7a7a7a74040a7a7a7a7a7f606a60070a2707070a270a2
a270707070708506f6a7a7a7a7a7f6f6f6f6f6f6404040f60694707070707070707070708616f6a7a7a740a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7b1b2
b206947070707000007070707070708406404040a7a7a7a7a7a703b1b2c0a7a7404003a7a7404040a7a7a7a7a7a7a7a7a7a7a7f6f606a60070701770707070a2
a270707070707406f6a7a7a7a7a7a7a7a7a7a7a7404040f60694d0d0d0d0d270707070708506f603a7a7a7a7a70340404003a7a7a7a7a7a7a7a7a740a7a7a7b1
c116967070701700707070707070707416404040a7a7a7a7a7a7a703b1c1a7a7404003a7a74040f6f6f6f6f6f6f6f6f6f6f6f6f6f606940070707070707070a2
a270701770707406f6a740a7a7a7a7a7a7a7a7a7a7a7a7f6069470701770d170707070707406f6f6f6f6f6f6f60340404003f6f6f6f6f6a7a7a7a7a7a7a74040
f606a67070707000007070707070708406f6a7a7a7a7a7a7a7a7a7a7a7a7a7a7a703a7a7a7a7a7f6c6060607060606060606070606c6940070707070177070a2
a270707070708616f640a7a7a7a7a7a7a7a7a7a7a7a7a7f60696707070a2a2700070177074b6060606060607060606060707060606b6f6a7a7a7a7a7a7f64040
f616967070707070707070707070708606f6a7f6e7f6f6f6f6f6f6f6f6f6f6f6f603f6f6a7b7a7f60626146466646644041464646604c50070707070707070a2
a2a27070707084064040a7a7a7a7a7a7a7a7a7a7a7a7a7f606a670707000a2a200707070b55404541466046644040464045404543606f7a740a7a7a7a7f6c606
06c6947070707070707070707070708606e7b7f6c606060606070606060606060606b6f6a7a7a7f60696707070707070707070701770707070707070707070a2
a2a2707070708406f6f6f6f6f6f6f6f6f6f6a7a703a7a7f61696707070707070707000a200707070707070d170707070707070707516f6a7a7a7a7a7a7f60626
0404c57070700000007070707070708506f6e7f6062614660466440404646644043606f6f6f6f6f60694707770707070707070707070707070a27070a27070a2
a2a27070707074b60606060606060606b6f6a703a703a7f606a670707070707070707070a2707070707070d170177070707070707034f787a7a7a7a7a7f60695
70707070701700000070000000000074b6060706c694707070707070707070707074b60606070606c694707070a27017707070707070707070705161707070a2
a27070177070b504545454146604043606f6a7a703a7a7b006a67070707017707070707070a27070a2d0d0d3707070707070707070709787a740a7a7a7f616a5
7070a2707070000000707070707070b56604666644c57070701770707017707070b504041464666466c57070a2707070707070707070707070705262707070a2
a27070707070707070707070d17070740640a7a7a7a7b0b20694707070707070707070c2d0a2d0a270a2707070707070707070707070a797a7a7a7a7a7f616a4
000070a27070707070707070707070707070707070707070707070707070707070707070707070707070707017707070701770707017707070a27070a27070a2
a27070707070701770c277d0777070740640f6f6f6f6b2b206947070707070707070707770a27070a2d2707070707070701770707035f7f6e7f6f6f6f6f606a4
70777070707070707070177070707070707070707070000000a2a2a2a2a2707070707070707070707017707070707070707070707070707070707070707070a2
a27070707070707070d1707070707074b606060606060606c6947070707070707070707070a2707070d17070707070707070707085b60606060706060606c694
70707070707070707070707070707070707070707070000000a2707070a27070701770707070707070707070707070707070707070707070707070707070a270
70a2a2a2a2a2a2a2a2a2a2a2a2a2a2b5040454146604660404c5a2a2a2a2a2a2a2a2a2a2a270a2a2a2a2a2a2a2a2a2a2a2a2a2a2b504146604664404046404c5
a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2707070a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a27070
""".replace('\n', '')

logo = """
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee3eeeeeeeeee
33333333e33eee333e33333333e33333333e33333333e33e33333333e33333333eeeeeee33333333e33333333e33333333e33333333e33333333e33eeee333ee
33333333e33ee333ee33333333e33333333e33333333e33e33333333e33333333eeeeeee33333333e33333333e33333333e33333333e33333333e333ee333eee
33eeee33e33e333eeeeeeeeeeeeeeeeee33eeeeeee33e33e33eeee33eeeeeeeeeeeeeeee33eeeeeee33eeee33eeeeeee33eeee33eeeeeeeeeeeeee333333eeee
33eeee33e33333eeee33333333e33333333e33333333e33e33eeee33e33333333eeeeeee33eeeeeee33eeee33e33333333eeee33eeee33333333eee3333eeeee
33eeee33e3333eeeee33333333e33333333e33333333e33e33eee333e33333333eeeeeee33eeeeeee33eeee33e33333333eeee33eeee33333333eee3333eeeee
33eeee33e333eeeeeeeeeeeeeee33e333eee33e333eee33e33ee333eeeeeeeeeeeeeeeee33eeeeeee33eeee33e33e333eeeeee33eeeeeeeeeeeeee333333eeee
33333333e33eeeeeee33333333e33ee333ee33ee333ee33e33e333eee33333333eeeeeee33333333e33333333e33ee333eeeee33eeee33333333e333ee333eee
33333333e3eeeeeeee33333333e33eee333e33eee333e33e33333eeee33333333eeeeeee33333333e33333333e33eee333eeee33eeee33333333333eeee33eee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee33eeeeeee33eeee3333eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee33eeeeeeeeeeeeeeeeeeeeeeeeee3eee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee3eeeeeeee3eeee333eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee3eeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee33eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee3eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
""".replace('\n', '')

In [11]:
# Define your map lists
map_1_2 = [map_1_2_top, map_1_2_bottom]
map_3_4 = [map_3_4_top, map_3_4_bottom, logo]


print("Processing Map 1_2:")
compressed_1_2 = process_map(map_1_2, 256, False)

print("Processing Map 3_4:")
compressed_3_4 = process_map(map_3_4, 128, True)

Processing Map 1_2:
Map piece 1:
Original length: 4096
Compressed length: 2005
Compression ratio: 0.49
Matches original: True

Map piece 2:
Original length: 3072
Compressed length: 1585
Compression ratio: 0.52
Matches original: True

Lengths of compressed pieces:
[2005, 1585]
Total compressed length: 3590

Compressed data:
0381c55000f808281c0e533b1d8a2073881cac762b150a05028c02c80f00728009154a2562b1da00c500c32612e034f0145808b4080a060c4c82fe0009703b381c9c0f4a080701e203c001800000ec029200aa0732180c07030180a45629c018e016a768089c0352006e6b2491cd901a6808dc0c24d64982fa1c40105d280190
a8af01c43841114e101e783fb0f01a0d150b61204710043d88be04379bcfc6f379bcfe6f0d8ac0804806140352004c6d8051810de7b301248e603713a01eb0035808981203450204021bcc05983e1436880000160f600a607219f808783fc1648da7e379fcfc6f3012a2ce47187fbc424c2a161ec428a22667e3c1e8f4793d1e
0f27a3d06c39086280394004e0285008236c030a0bc404301648e60379ee01eb01333dc1fd0de7b0201205a707f33481c0e7783fa0740110430381cfe7982891e09a5e3d1e8fc7b3798623c477068345b